## 参照ゲノム配列なしのRNA-seq解析手法

### 入力配列の品質管理

In [ ]:
!conda install -c bioconda trim-galore -y

In [ ]:
!pwd

In [ ]:
%cd trim_galore/

In [ ]:
#trim galoreの実行
!time for drr in DRR068893 DRR068894 DRR068895; do trim_galore --fastqc -o trim_galore/ --paired $drr.sra_1.fastq.gz $drr.sra_2.fastq.gz ; done 

In [ ]:
#中身の確認
!ls trim_galore/

In [ ]:
#fastqcのレポートを見てみる
!open trim_galore/DRR068893.sra_1_val_1_fastqc.html

### Trinityの実行

In [ ]:
#condaではなく、Homebrewで。
!brew install trinity
#Javaが必要ならば
#!brew cask install Java 

In [ ]:
#fastqファイルのヘッダ改変スクリプトを取得
!curl -O curl -O https://raw.githubusercontent.com/bonohu/denovoTA/master/for_trinity.pl
#3種類*2（ペア）＝6つのファイルに対してスクリプトを実行する
!for drr in DRR068893 DRR068894 DRR068895; do echo "processing ${drr}_1" && pigz -dc trim_galore/$drr.sra_1_val_1.fq.gz | perl for_trinity.pl 1 | pigz -c > ${drr}_1.fq.gz && echo "processing ${drr}_2" && pigz -dc trim_galore/$drr.sra_2_val_2.fq.gz | perl for_trinity.pl 2 | pigz -c > ${drr}_2.fq.gz ; done

In [ ]:
#trinityの実行
#これは2、3日間かかる。
!Trinity --seqType fq --trimmomatic --left DRR068893_1.fq.gz,DRR068894_1.fq.gz,DRR068895_1.fq.gz --right DRR068893_2.fq.gz,DRR068894_2.fq.gz,DRR068895_2.fq.gz --max_memory 500G --CPU 20 

In [1]:
!less trinity_out_dir/trinity.fasta

>TRINITY_DN2_c0_g1_i1 len=4006 path=[0:0-4005]
GAAGAGCCTCGGCCACCTCCAGGTTGTTATACCCGGCAGCCAAATGCAAGGGCGTGGAGTTACGTCCGTGCGCGTCCCGG:

In [2]:
#得られた配列クラスタ数をカウント
!grep -c ^\> trinity_out_dir/trinity.fasta

85999


### 発現定量

In [10]:
#salmonを使った発現定量
!chmod 755 ./align_and_estimate_abundance.pl
!time ./align_and_estimate_abundance.pl --thread_count 4 --transcripts trinity_out_dir/Trinity.fasta --seqType fq --samples_file samples.txt --est_method salmon --salmon_add_opts "--validateMappings" --prep_reference

m 11,500,000 fragments
processed 12,000,000 fragments
processed 12,500,000 fragments
processed 13,000,000 fragments
processed 13,500,000 fragments
processed 14,000,000 fragments
processed 14,500,000 fragments
processed 15,000,000 fragments
processed 15,500,000 fragments
processed 16,000,000 fragments
processed 16,500,000 fragments
hits: 24,017,838, hits per frag:  1.45639







[2019-10-16 12:29:57.666] [jointLog] [info] Computed 137,477 rich equivalence classes for further processing
[2019-10-16 12:29:57.666] [jointLog] [info] Counted 9,999,710 total reads in the equivalence classes 
[2019-10-16 12:29:57.672] [jointLog] [info] Number of mappings discarded because of alignment score : 1,683,505
[2019-10-16 12:29:57.672] [jointLog] [info] Number of fragments entirely discarded because of alignment score : 201,730
[2019-10-16 12:29:57.672] [jointLog] [info] Number of fragments discarded because they are best-mapped to decoys : 0
[2019-10-16 12:29:57.672] [jointLog] [info] Number of frag

In [15]:
#確認
!ls | grep testis

testis_rep1
testis_rep2
testis_rep3


In [16]:
#定量結果の確認
!less testis_rep1/quant.sf

Name    Length  EffectiveLength TPM     NumReads
TRINITY_DN2_c0_g1_i1    4006    3795.679        49.550437       1646.000
TRINITY_DN13_c0_g1_i3   7244    7033.679        8.728194        537.279
TRINITY_DN13_c0_g1_i5   9077    8866.679        0.000000        0.000
TRINITY_DN13_c0_g1_i1   9078    8867.679        6.485037        503.286
TRINITY_DN13_c0_g1_i2   6924    6713.679        20.603060       1210.557
TRINITY_DN13_c0_g1_i4   9077    8866.679        16.777082       1301.878
TRINITY_DN13_c0_g2_i2   2811    2600.679        2.192350        49.899
TRINITY_DN13_c0_g2_i1   2620    2409.679        61.032427       1287.101
TRINITY_DN13_c1_g1_i1   214     37.677  0.000000        0.000
TRINITY_DN13_c1_g1_i2   264     73.480  1.555035        1.000
TRINITY_DN1_c0_g1_i1    2567    2356.679        211.685159      4366.000
TRINITY_DN1_c1_g1_i2    2086    1875.679        0.873358        14.337
TRINITY_DN1_c1_g1_i6    1815    1604.679        0.521389        7.322
TRINITY_DN1_c1_g1_i1    1529    1318

### タンパク質コード配列の予測

3.4章「ドメイン解析」でのTransDecoderを使用できる。